# STM32 Model Analysis (Mock Version)

Analyze TFLite models without STM32TFLM executable

In [1]:
from mock_stm32_analyzer import analyze_trained_models_mock, compare_with_constraints
import os

print("Using mock STM32 analyzer (estimates only)")

Using mock STM32 analyzer (estimates only)


In [4]:
# Configuration
MODELS_DIR = "./trained_models"  # Change to your models directory
MAX_RAM = 40960    # 40 KB
MAX_FLASH = 131072 # 128 KB

print(f"Models directory: {MODELS_DIR}")
print(f"Constraints: {MAX_RAM/1024:.0f} KB RAM, {MAX_FLASH/1024:.0f} KB Flash")

Models directory: ./trained_models
Constraints: 40 KB RAM, 128 KB Flash


In [5]:
# Analyze models
results = analyze_trained_models_mock(MODELS_DIR)
print(f"\nAnalyzed {len(results)} models")

⚠️ Using mock STM32 analyzer (STM32TFLM not available)
Found 1 TFLite models

Analyzing: resulting_architecture_k_8_c_0.tflite
----------------------------------------
📊 Model Size: 4.48 KB
💾 Flash Usage: 5.37 KB (estimated)
🔧 RAM Usage: 87.95 KB (estimated)

Analyzed 1 models


c:\Users\Chakkaphan\anaconda3\envs\tinyml\lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [6]:
# Check constraints
if results:
    feasible = compare_with_constraints(results, MAX_RAM, MAX_FLASH)
    
    if feasible:
        best = min(feasible, key=lambda x: x['ram_bytes'])
        print(f"\n🏆 Best model: {best['model_name']}")
        print(f"Flash: {best['flash_bytes']/1024:.1f} KB")
        print(f"RAM: {best['ram_bytes']/1024:.1f} KB")
    else:
        print("\n⚠️ No models meet constraints")
else:
    print("No models found")


🎯 CONSTRAINT ANALYSIS
Max RAM: 40.0 KB
Max Flash: 128.0 KB
----------------------------------------
❌ resulting_architecture_k_8_c_0.tflite
   Flash: 5.37 KB (OK)
   RAM: 87.95 KB (EXCEED)

✅ Feasible models: 0/1

⚠️ No models meet constraints


In [ ]:
# Export results
if results:
    import pandas as pd
    
    df = pd.DataFrame(results)
    df['flash_kb'] = df['flash_bytes'] / 1024
    df['ram_kb'] = df['ram_bytes'] / 1024
    df['feasible'] = (df['flash_bytes'] <= MAX_FLASH) & (df['ram_bytes'] <= MAX_RAM)
    
    print("\n📊 Results Summary:")
    print(df[['model_name', 'flash_kb', 'ram_kb', 'feasible']].to_string(index=False))
    
    df.to_csv('mock_stm32_results.csv', index=False)
    print("\n💾 Saved to mock_stm32_results.csv")